# Extracting models performances
Extract all the models performances details from the exported json files and present all the results in a pandas DataFrame to format in Markdown 

In [24]:
# !pip install tabulate==0.8.10 pandas==1.4.1

In [27]:
import os
import json
import pandas as pd

In [39]:
path_to_json = '../scripts/perf' # perf output folder
perf_data = []
for file_name in [file for file in os.listdir(path_to_json) if file.endswith('.json')]:
    with open(os.path.join(path_to_json, file_name)) as json_file:
        data = json.load(json_file)
        perf_data.append(data)

In [41]:
df = pd.DataFrame.from_records(perf_data, index=list(range(len(perf_data))))
df = df.sort_values(by="People count", ascending=True)
res = df.drop("Detection confidence threshold", axis=1)
# print(res.to_markdown())
res.reset_index(drop=True)

,people count,AVG inference time (sec.),Model Name,Completion Time (Sec.),Floating-point precision,Size
0,8,0.021,pedestrian-detection-adas-0002,131.864,FP32,88.00 B
1,8,0.021,pedestrian-detection-adas-0002,132.570,FP16,88.00 B
2,16,0.016,ssdlite_mobilenet_v2,126.432,FP16,88.00 B
3,16,0.016,ssdlite_mobilenet_v2,125.300,FP32,88.00 B
4,26,0.071,ssd_mobilenet_v2_coco,231.025,FP32,88.00 B
5,27,0.022,ssd_mobilenet_v2_coco,132.712,FP16,88.00 B
6,31,1.356,person-detection-retail-0013,2024.693,FP16,88.00 B
7,33,0.017,person-detection-retail-0013,130.609,FP32,88.00 B


In [42]:
path_to_json = '../scripts/perf_latest_26.08/'
perf_data = []
for file_name in [file for file in os.listdir(path_to_json) if file.endswith('.json')]:
    with open(os.path.join(path_to_json, file_name)) as json_file:
        data = json.load(json_file)
        perf_data.append(data)
df = pd.DataFrame.from_records(perf_data, index=list(range(len(perf_data))))
df = df.sort_values(by="People count", ascending=True)
res = df.drop("Detection confidence threshold", axis=1)
res =  res.drop("Size", axis=1)
res.reset_index(drop=True, inplace=True)
print(path_to_json)
print(res.to_markdown())
res

../scripts/perf_latest_26.08/
|    |   people count |   AVG inference time (sec.) | Model Name                     |   Completion Time (Sec.) | Floating-point precision   |
|---:|---------------:|----------------------------:|:-------------------------------|-------------------------:|:---------------------------|
|  0 |              8 |                       0.021 | pedestrian-detection-adas-0002 |                  131.864 | FP32                       |
|  1 |              8 |                       0.021 | pedestrian-detection-adas-0002 |                  132.57  | FP16                       |
|  2 |             16 |                       0.016 | ssdlite_mobilenet_v2           |                  126.432 | FP16                       |
|  3 |             16 |                       0.016 | ssdlite_mobilenet_v2           |                  125.3   | FP32                       |
|  4 |             26 |                       0.071 | ssd_mobilenet_v2_coco          |                  231.025 

,people count,AVG inference time (sec.),Model Name,Completion Time (Sec.),Floating-point precision
0,8,0.021,pedestrian-detection-adas-0002,131.864,FP32
1,8,0.021,pedestrian-detection-adas-0002,132.570,FP16
2,16,0.016,ssdlite_mobilenet_v2,126.432,FP16
3,16,0.016,ssdlite_mobilenet_v2,125.300,FP32
4,26,0.071,ssd_mobilenet_v2_coco,231.025,FP32
5,27,0.022,ssd_mobilenet_v2_coco,132.712,FP16
6,31,1.356,person-detection-retail-0013,2024.693,FP16
7,33,0.017,person-detection-retail-0013,130.609,FP32


# Models Performance comparison

| Model Name | Implementation | Complexity (GFLOPs)  | Size(MParams) | Accuracy (%) | Input Shape [BxCxHxW]| Avg Inference Time(ms)  | Completion Time(s)  | #People Detected |
-------------------------------------------------------------------------------------------------------------------------------- |----|--------------------- |----------- |------ |-------- |--------- |------ |--------------- |
[ssd_mobilenet_v2_coco](https://docs.openvino.ai/2021.4/omz_models_model_ssd_mobilenet_v2_coco.html) | Tensorflow|3.775 | 16.818 | (coco precision) 24.9452% | [1x3x300x300]|0.012| 105.35 | 28 |
[pedestrian-detection-adas-0002](https://docs.openvino.ai/2021.4/omz_models_model_pedestrian_detection_adas_0002.html) |	Caffe|2.836 | 1.165 | AP 88% |  [1x3x384x672] | inf|total|count|
[person-detection-retail-0013](https://docs.openvino.ai/2021.4/omz_models_model_person_detection_retail_0013.html)|Caffe|2.300 | 0.723 | AP 88.62%| [1x3x320x544]| 0.009 | 102.248| 9|
[ssdlite_mobilenet_v2](https://docs.openvino.ai/2021.4/omz_models_model_ssdlite_mobilenet_v2.html)| TensorFlow| 1.525

| 4.475 |24.2946%| [1x3x480x864]| inf|time|count|

In [43]:
res = res.groupby('Floating-point precision').agg(
    {'AVG inference time (sec.)': ['min', 'max'],
     'People count': 'max', 'Model Name': 'first'})
res

AVG inference time (sec.)        people count  \
                                               min    max          max   
Floating-point precision                                                 
FP16                                         0.016  1.356           31   
FP32                                         0.016  0.071           33   

                                              Model Name  
                                                   first  
Floating-point precision                                  
FP16                      pedestrian-detection-adas-0002  
FP32                      pedestrian-detection-adas-0002